In [41]:
import pathlib

path=str(pathlib.Path().resolve())+"/"

In [42]:
import pandas as pd


df = pd.read_csv(path+"reviews.csv")

In [43]:
df=df[:1000]

In [98]:
import torch
import transformers
from transformers import pipeline
from transformers import AutoTokenizer

device = 0 if torch.cuda.is_available() else -1
transformers.utils.logging.set_verbosity_error()

nlp = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", device=device)
tokenize = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [117]:
import numpy as np

def tokenizer(x):
    #tokenizer = " ".join(x.split(" ")).split("\n")
    #print(tokenizer.count(" "))
    
    if len(tokenize.tokenize(x))>512:
        segments = [x[i:i + 512] for i in range(0, len(x.split(" ")), 512)]
        results = []
        for segment in segments:
            predict = nlp(segment)[0]
            results.append((predict['label'].split(" ")[0], round(predict['score'], 4)))
        labels=np.array(results)[:,0]
        resultados=np.array(results)[:,1]
        return f"{np.average(int(labels)) if len(results)>1 else labels[0]} stars",np.average(results) if len(results)>1 else resultados[0]
    
    predict = nlp(x)[0]
    return predict['label'], round(predict['score'], 4)

x = df['review'].apply(tokenizer)


In [ ]:
for i in x:
    print(i)

In [119]:
import numpy as np
l = np.array(x.tolist())
df['label'],df['label_score'] = l[:,0],l[:,1]

,id_review,review,title,score,likes,id_nick,service,date,platform,label,label_score
